In [ ]:
import requests
import re

In [ ]:
def get_domain_rules(domain):
    res = requests.get(f'{domain}/robots.txt')
    if res.status_code == 200:
        return res.text
    return None

rules = get_domain_rules('https://-.com')
rules

In [ ]:
def filter_disallow_rules(rules):
    rulelist = rules.splitlines()
    disallows = []
    for rule in rulelist:
        if rule.startswith('Disallow: '):
            disallows.append(rule.split(': ')[1])
    return disallows

In [ ]:
disallows = filter_disallow_rules(rules)
disallows

In [ ]:
def filter_paths(paths, filters):
    res = paths[:]
    for f in filters:
        res = list(filter(lambda p: not re.search(f, p), res))
    return res

In [ ]:
filters = [
    r'[*]', # * is used for generalization, not a file
    r'[=]', # = is used for paths with specific arguments
]
disallows = filter_paths(disallows, filters)
disallows

In [ ]:
import shutil
import os

def clean_target(path):
    shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)
    
clean_target('target')

In [ ]:
def download_paths(domain, paths, local='target'):
    for path in paths:
        url = f'{domain}{path}'
        res = requests.get(url)
        if res.status_code == 200:
            print(f'[+] Found {url}')
            with open(f'{local}/{path.replace("/", "--").strip("-")}', 'w') as f:
                f.write(res.text)
        else:
            print(f'[-] {url} returned {res.status_code}')

In [ ]:
download_paths('https://-.com', disallows)